
### Sistema de Recomendação de Produtos Agrícolas do DF

Projeto 1 - Introdução à Inteligência Artificial

Professor Díbio - UnB 2025/1

### Alunos

- Raul Myron Silva Amorim - 200049712
- Lucca Santos Aguilar - 241024221
- Thais Aragão Bianchini - 221007205

Este notebook implementa um sistema de recomendação para
pequenos produtores rurais a consumidores no Distrito Federal.


In [48]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import folium
import json
from datetime import datetime
import random
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
import warnings
warnings.filterwarnings('ignore')

In [49]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000)

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

In [50]:
"""
Dados extraídos manualmente dos documentos fornecidos:
- 17 Associações/Cooperativas com coordenadas geográficas
- Produtos oferecidos por cada uma (baseado nos editais do GDF)
- Regiões de atuação no DF
"""

# Lista completa de produtos do escopo do projeto
produtos_escopo = [
    'Alface', 'Mandioca', 'Tomate', 'Repolho', 'Batata', 'Cebola', 'Couve',
    'Chuchu', 'Morango', 'Pimentão', 'Brócolis', 'Abóbora', 'Berinjela',
    'Beterraba', 'Pepino', 'Cenoura', 'Quiabo', 'Agrião', 'Jiló', 'Gengibre',
    'Abacate', 'Goiaba', 'Banana', 'Limão', 'Tangerina', 'Maracujá', 'Manga',
    'Lichia', 'Uva', 'Atemóia', 'Cajamanga', 'Graviola', 'Coco', 'Pitaia', 'Mamão'
]

# Mapeamento entre nomes dos produtos nos documentos e produtos do escopo
mapeamento_produtos = {
    'Abóbora Japonesa': 'Abóbora',
    'Abobrinha Italiana': 'Abóbora',
    'Acelga': 'Couve',
    'Alface Americana': 'Alface',
    'Cebolinha Comum': 'Cebola',
    'Coentro': 'Couve',
    'Couve-Flor': 'Brócolis',
    'Couve Manteiga': 'Couve',
    'Espinafre': 'Couve',
    'Hortelã': 'Agrião',
    'Manjericão': 'Agrião',
    'Pepino Comum': 'Pepino',
    'Pimentão Verde': 'Pimentão',
    'Repolho Verde': 'Repolho',
    'Repolho Roxo': 'Repolho',
    'Salsa': 'Agrião',
    'Batata Doce': 'Batata',
    'Brócolis Cabeça Única (Japonês)': 'Brócolis',
    'Inhame': 'Batata',
    'Limão Tahiti': 'Limão',
    'Milho Verde': 'Pepino',
    'Tangerina Ponkan': 'Tangerina',
    'Vagem': 'Pepino',
    'Banana Prata': 'Banana',
    'Alho': 'Cebola'
}

# Dados das 17 associações/cooperativas
associacoes_data = [
    {
        'id': 1,
        'nome': 'AFECA - Assentamento 15 de Agosto',
        'latitude': -15.911319,
        'longitude': -47.721311,
        'regioes': ['São Sebastião'],
        'produtos_originais': ['Abóbora Japonesa', 'Abobrinha Italiana', 'Acelga', 'Alface Americana', 
                              'Cebolinha Comum', 'Chuchu', 'Coentro', 'Couve-Flor', 'Couve Manteiga', 
                              'Espinafre', 'Hortelã', 'Manjericão', 'Maracujá', 'Pepino Comum', 
                              'Pimentão Verde', 'Repolho Verde', 'Repolho Roxo', 'Salsa', 'Tomate']
    },
    {
        'id': 2,
        'nome': 'AGRIFAM - Agricultura Familiar DF',
        'latitude': -15.932992,
        'longitude': -48.040909,
        'regioes': ['Taguatinga', 'Gama', 'Santa Maria'],
        'produtos_originais': ['Abacate', 'Abóbora Japonesa', 'Abobrinha Italiana', 'Acelga', 
                              'Alface Americana', 'Alho', 'Batata Doce', 'Beterraba', 
                              'Brócolis Cabeça Única (Japonês)', 'Cebola', 'Cebolinha Comum', 
                              'Cenoura', 'Chuchu', 'Coentro', 'Couve-Flor', 'Couve Manteiga', 
                              'Espinafre', 'Goiaba', 'Hortelã', 'Inhame', 'Limão Tahiti', 
                              'Manjericão', 'Maracujá', 'Milho Verde', 'Pepino Comum', 
                              'Pimentão Verde', 'Repolho Verde', 'Repolho Roxo', 'Salsa', 
                              'Tangerina Ponkan', 'Tomate', 'Vagem', 'Banana Prata']
    },
    {
        'id': 3,
        'nome': 'AMISTA - Agricultores Orgânicos',
        'latitude': -15.782057,
        'longitude': -47.470923,
        'regioes': ['Santa Maria'],
        'produtos_originais': ['Abacate', 'Abóbora Japonesa', 'Abobrinha Italiana', 'Batata Doce', 
                              'Beterraba', 'Brócolis Cabeça Única (Japonês)', 'Cebolinha Comum', 
                              'Cenoura', 'Chuchu', 'Coentro', 'Couve-Flor', 'Couve Manteiga', 
                              'Inhame', 'Limão Tahiti', 'Maracujá', 'Pepino Comum', 'Pimentão Verde', 
                              'Repolho Verde', 'Repolho Roxo', 'Salsa', 'Tomate', 'Vagem']
    },
    {
        'id': 4,
        'nome': 'ASPAF - Produtores Agricultura Familiar',
        'latitude': -15.940404,
        'longitude': -47.737964,
        'regioes': ['Guará', 'Núcleo Bandeirante', 'Plano Piloto'],
        'produtos_originais': ['Goiaba', 'Morango', 'Tangerina Ponkan', 'Abacate', 'Abóbora Japonesa', 
                              'Abobrinha Italiana', 'Acelga', 'Alface Americana', 'Batata Doce', 
                              'Beterraba', 'Brócolis Cabeça Única (Japonês)', 'Cebola', 
                              'Cebolinha Comum', 'Cenoura', 'Chuchu', 'Coentro', 'Couve-Flor', 
                              'Couve Manteiga', 'Espinafre', 'Hortelã', 'Inhame', 'Limão Tahiti', 
                              'Manjericão', 'Maracujá', 'Milho Verde', 'Pepino Comum', 
                              'Pimentão Verde', 'Repolho Verde', 'Repolho Roxo', 'Salsa', 
                              'Tomate', 'Vagem', 'Banana Prata']
    },
    {
        'id': 5,
        'nome': 'ASPAG - Alexandre Gusmão',
        'latitude': -15.726869,
        'longitude': -48.189019,
        'regioes': ['Alexandre Gusmão', 'Brazlândia'],
        'produtos_originais': ['Abacate', 'Abóbora Japonesa', 'Abobrinha Italiana', 'Acelga', 
                              'Alface Americana', 'Batata Doce', 'Beterraba', 
                              'Brócolis Cabeça Única (Japonês)', 'Cebolinha Comum', 'Cenoura', 
                              'Chuchu', 'Coentro', 'Couve-Flor', 'Couve Manteiga', 'Espinafre', 
                              'Goiaba', 'Hortelã', 'Inhame', 'Limão Tahiti', 'Manjericão', 
                              'Maracujá', 'Milho Verde', 'Pepino Comum', 'Pimentão Verde', 
                              'Repolho Verde', 'Repolho Roxo', 'Salsa', 'Tangerina Ponkan', 
                              'Tomate', 'Vagem', 'Banana Prata']
    },
    {
        'id': 6,
        'nome': 'ASPHOR - Hortigranjeiros DF',
        'latitude': -15.791442,
        'longitude': -47.948464,
        'regioes': ['Plano Piloto', 'Gama', 'Santa Maria'],
        'produtos_originais': ['Abacate', 'Abóbora Japonesa', 'Abobrinha Italiana', 'Acelga', 
                              'Alface Americana', 'Batata Doce', 'Beterraba', 
                              'Brócolis Cabeça Única (Japonês)', 'Cebola', 'Cebolinha Comum', 
                              'Cenoura', 'Chuchu', 'Coentro', 'Couve-Flor', 'Couve Manteiga', 
                              'Espinafre', 'Goiaba', 'Hortelã', 'Inhame', 'Limão Tahiti', 
                              'Manjericão', 'Maracujá', 'Milho Verde', 'Morango', 'Pepino Comum', 
                              'Pimentão Verde', 'Repolho Verde', 'Repolho Roxo', 'Salsa', 
                              'Tomate', 'Vagem']
    },
    {
        'id': 7,
        'nome': 'ASPROC - Orgânicos e Convencionais',
        'latitude': -15.777904,
        'longitude': -48.120988,
        'regioes': ['Ceilândia', 'Recanto das Emas', 'Samambaia', 'Paranoá', 'Planaltina'],
        'produtos_originais': ['Goiaba', 'Tangerina Ponkan', 'Morango']
    },
    {
        'id': 8,
        'nome': 'ASPRONTE - Novo Horizonte',
        'latitude': -15.603130,
        'longitude': -48.113516,
        'regioes': ['Brazlândia', 'Ceilândia', 'Recanto Das Emas'],
        'produtos_originais': ['Morango', 'Abacate', 'Abobrinha Italiana', 'Acelga', 
                              'Alface Americana', 'Batata Doce', 'Beterraba', 
                              'Brócolis Cabeça Única (Japonês)', 'Cebolinha Comum', 'Cenoura', 
                              'Chuchu', 'Coentro', 'Couve-Flor', 'Couve Manteiga', 'Espinafre', 
                              'Goiaba', 'Hortelã', 'Pepino Comum', 'Pimentão Verde', 
                              'Repolho Verde', 'Salsa', 'Tomate', 'Vagem']
    },
    {
        'id': 9,
        'nome': 'ASTRAF - Assentamento Chapadinha',
        'latitude': -15.542238,
        'longitude': -48.029995,
        'regioes': ['PAD-DF', 'Planaltina'],
        'produtos_originais': ['Abacate', 'Abóbora Japonesa', 'Abobrinha Italiana', 'Acelga', 
                              'Alface Americana', 'Alho', 'Batata Doce', 'Beterraba', 
                              'Brócolis Cabeça Única (Japonês)', 'Cebola', 'Cebolinha Comum', 
                              'Cenoura', 'Chuchu', 'Coentro', 'Couve-Flor', 'Couve Manteiga', 
                              'Espinafre', 'Hortelã', 'Inhame', 'Limão Tahiti', 'Manjericão', 
                              'Maracujá', 'Milho Verde', 'Pepino Comum', 'Pimentão Verde', 
                              'Repolho Verde', 'Repolho Roxo', 'Salsa', 'Tomate', 'Vagem', 
                              'Banana Prata']
    },
    {
        'id': 10,
        'nome': 'COOPBRASIL - Agricultura Familiar',
        'latitude': -15.918585,
        'longitude': -48.082794,
        'regioes': ['Recanto das Emas', 'Gama', 'Samambaia', 'Núcleo Bandeirante', 
                   'Planaltina', 'Brazlândia', 'Ceilândia'],
        'produtos_originais': ['Alho', 'Abóbora Japonesa', 'Cebola', 'Inhame', 'Limão Tahiti', 
                              'Manjericão', 'Maracujá', 'Milho Verde', 'Repolho Roxo', 
                              'Banana Prata', 'Abacate', 'Abobrinha Italiana', 'Acelga', 
                              'Alface Americana', 'Batata Doce', 'Beterraba', 
                              'Brócolis Cabeça Única (Japonês)', 'Cebolinha Comum', 'Cenoura', 
                              'Chuchu', 'Coentro', 'Couve-Flor', 'Couve Manteiga', 'Espinafre', 
                              'Goiaba', 'Hortelã', 'Pepino Comum', 'Pimentão Verde', 
                              'Repolho Verde', 'Salsa', 'Tomate', 'Vagem', 'Morango']
    },
    {
        'id': 11,
        'nome': 'Cooper-Horti - Buriti Vermelho',
        'latitude': -15.898749,
        'longitude': -47.409203,
        'regioes': ['Paranoá', 'PAD-DF'],
        'produtos_originais': ['Abacate', 'Abóbora Japonesa', 'Abobrinha Italiana', 'Batata Doce', 
                              'Beterraba', 'Cenoura', 'Chuchu', 'Couve Manteiga', 'Inhame', 
                              'Limão Tahiti', 'Maracujá', 'Pepino Comum', 'Pimentão Verde', 
                              'Repolho Verde', 'Tangerina Ponkan']
    },
    {
        'id': 12,
        'nome': 'Prorural - Planaltina GO',
        'latitude': -15.445505,
        'longitude': -47.619013,
        'regioes': ['Planaltina', 'Plano Piloto', 'Paranoá'],
        'produtos_originais': ['Abacate', 'Abóbora Japonesa', 'Abobrinha Italiana', 'Acelga', 
                              'Alface Americana', 'Alho', 'Batata Doce', 'Beterraba', 
                              'Brócolis Cabeça Única (Japonês)', 'Cebola', 'Cebolinha Comum', 
                              'Cenoura', 'Chuchu', 'Coentro', 'Couve-Flor', 'Couve Manteiga', 
                              'Espinafre', 'Goiaba', 'Hortelã', 'Inhame', 'Limão Tahiti', 
                              'Manjericão', 'Maracujá', 'Milho Verde', 'Pepino Comum', 
                              'Pimentão Verde', 'Repolho Verde', 'Repolho Roxo', 'Salsa', 
                              'Tomate', 'Vagem', 'Banana Prata']
    },
    {
        'id': 13,
        'nome': 'Coopebraz - Brazlândia',
        'latitude': -15.602622,
        'longitude': -48.113771,
        'regioes': ['Brazlândia', 'Taguatinga', 'Samambaia', 'Recanto Das Emas'],
        'produtos_originais': ['Morango']
    },
    {
        'id': 14,
        'nome': 'Coopermista - Agricultura Familiar',
        'latitude': -15.764000,
        'longitude': -47.493246,
        'regioes': ['Planaltina'],
        'produtos_originais': ['Abacate', 'Abóbora Japonesa', 'Abobrinha Italiana', 'Batata Doce', 
                              'Beterraba', 'Cenoura', 'Couve Manteiga', 'Limão Tahiti', 
                              'Maracujá', 'Pepino Comum', 'Repolho Verde', 'Repolho Roxo', 
                              'Tangerina Ponkan']
    },
    {
        'id': 15,
        'nome': 'Rede Terra - Ecológicos do Cerrado',
        'latitude': -16.784921,
        'longitude': -47.600542,
        'regioes': ['Santa Maria', 'Gama'],
        'produtos_originais': ['Acelga', 'Alface Americana', 'Alho', 'Cebola', 'Espinafre', 
                              'Goiaba', 'Hortelã', 'Manjericão', 'Milho Verde']
    },
    {
        'id': 16,
        'nome': 'Cootaquara - Planaltina',
        'latitude': -15.632824,
        'longitude': -47.522650,
        'regioes': ['Planaltina', 'Ceilândia'],
        'produtos_originais': ['Abacate', 'Abóbora Japonesa', 'Abobrinha Italiana', 
                              'Alface Americana', 'Batata Doce', 'Beterraba', 
                              'Brócolis Cabeça Única (Japonês)', 'Cebolinha Comum', 'Cenoura', 
                              'Chuchu', 'Coentro', 'Couve-Flor', 'Couve Manteiga', 'Espinafre', 
                              'Inhame', 'Limão Tahiti', 'Maracujá', 'Milho Verde', 'Pepino Comum', 
                              'Pimentão Verde', 'Repolho Verde', 'Repolho Roxo', 'Salsa', 
                              'Tomate', 'Vagem', 'Banana Prata']
    },
    {
        'id': 17,
        'nome': 'Cooperbrasília - Serviços Ambientais',
        'latitude': -15.918543,
        'longitude': -48.082816,
        'regioes': ['Sobradinho', 'São Sebastião'],
        'produtos_originais': ['Abacate', 'Abóbora Japonesa', 'Abobrinha Italiana', 'Acelga', 
                              'Alface Americana', 'Alho', 'Batata Doce', 'Beterraba', 
                              'Brócolis Cabeça Única (Japonês)', 'Cebola', 'Cebolinha Comum', 
                              'Cenoura', 'Chuchu', 'Coentro', 'Couve-Flor', 'Couve Manteiga', 
                              'Espinafre', 'Goiaba', 'Hortelã', 'Inhame', 'Limão Tahiti', 
                              'Manjericão', 'Maracujá', 'Milho Verde', 'Morango', 'Pepino Comum', 
                              'Pimentão Verde', 'Repolho Verde', 'Repolho Roxo', 'Salsa', 
                              'Tangerina Ponkan', 'Tomate', 'Vagem', 'Banana Prata']
    }
]

In [51]:
# Converter para DataFrame e mapear produtos
df_associacoes = pd.DataFrame(associacoes_data)

# Função para mapear produtos originais para produtos do escopo
def mapear_produtos(produtos_lista):
    produtos_mapeados = []
    for produto in produtos_lista:
        if produto in mapeamento_produtos:
            produto_mapeado = mapeamento_produtos[produto]
            if produto_mapeado in produtos_escopo and produto_mapeado not in produtos_mapeados:
                produtos_mapeados.append(produto_mapeado)
        elif produto in produtos_escopo and produto not in produtos_mapeados:
            produtos_mapeados.append(produto)
    return produtos_mapeados

# Aplicar mapeamento
df_associacoes['produtos'] = df_associacoes['produtos_originais'].apply(mapear_produtos)

# Adicionar características adicionais
np.random.seed(101)
df_associacoes['organico_principal'] = np.random.choice([True, False], size=len(df_associacoes), p=[0.3, 0.7])
df_associacoes['avaliacao_media'] = np.random.uniform(3.5, 5.0, size=len(df_associacoes))
df_associacoes['num_avaliacoes'] = np.random.randint(20, 200, size=len(df_associacoes))
df_associacoes['preco_medio_relativo'] = np.random.choice(['Baixo', 'Médio', 'Alto'], 
                                                          size=len(df_associacoes), 
                                                          p=[0.3, 0.5, 0.2])

# Ajustar algumas associações específicas como orgânicas
df_associacoes.loc[df_associacoes['nome'].str.contains('Orgânicos|Ecológicos'), 'organico_principal'] = True

print(f"Dados de {len(df_associacoes)} associações/cooperativas carregados com sucesso!")
print(f"\nTotal de produtos únicos no escopo: {len(produtos_escopo)}")
print(f"Total de produtos após mapeamento: {df_associacoes['produtos'].apply(len).sum()}")
print("\nPrimeiras 3 associações:")
print(df_associacoes[['id', 'nome', 'regioes', 'produtos']].head(3))

Dados de 17 associações/cooperativas carregados com sucesso!

Total de produtos únicos no escopo: 35
Total de produtos após mapeamento: 255

Primeiras 3 associações:
   id                               nome                          regioes                                           produtos
0   1  AFECA - Assentamento 15 de Agosto                  [São Sebastião]  [Abóbora, Couve, Alface, Cebola, Chuchu, Bróco...
1   2  AGRIFAM - Agricultura Familiar DF  [Taguatinga, Gama, Santa Maria]  [Abacate, Abóbora, Couve, Alface, Cebola, Bata...
2   3    AMISTA - Agricultores Orgânicos                    [Santa Maria]  [Abacate, Abóbora, Batata, Beterraba, Brócolis...


In [52]:
dados_taco = {
    'Abacate': {'id': 163, 'umidade': 83.8, 'energia_kcal': 96, 'proteina_g': 1.2, 
                'lipidios_g': 8.4, 'carboidratos_g': 6.0, 'fibra_g': 6.3, 'calcio_mg': 8, 
                'vitamina_c_mg': 15},
    'Abóbora': {'id': 71, 'umidade': 93.9, 'energia_kcal': 19, 'proteina_g': 1.1, 
                'lipidios_g': 0.1, 'carboidratos_g': 4.3, 'fibra_g': 1.4, 'calcio_mg': 15, 
                'vitamina_c_mg': 20},
    'Agrião': {'id': 74, 'umidade': 93.2, 'energia_kcal': 21, 'proteina_g': 1.4, 
                'lipidios_g': 0.1, 'carboidratos_g': 4.6, 'fibra_g': 1.1, 'calcio_mg': 43, 
                'vitamina_c_mg': 10},
    'Alface': {'id': 77, 'umidade': 97.2, 'energia_kcal': 9, 'proteina_g': 0.6, 
               'lipidios_g': 0.1, 'carboidratos_g': 1.7, 'fibra_g': 1.0, 'calcio_mg': 14, 
               'vitamina_c_mg': 6},
    'Batata': {'id': 89, 'umidade': 69.5, 'energia_kcal': 118, 'proteina_g': 1.3, 
               'lipidios_g': 0.1, 'carboidratos_g': 28.2, 'fibra_g': 2.6, 'calcio_mg': 21, 
               'vitamina_c_mg': 17},
    'Beterraba': {'id': 98, 'umidade': 86.0, 'energia_kcal': 49, 'proteina_g': 1.9, 
                  'lipidios_g': 0.1, 'carboidratos_g': 11.1, 'fibra_g': 3.4, 'calcio_mg': 18, 
                  'vitamina_c_mg': 24},
    'Brócolis': {'id': 101, 'umidade': 91.2, 'energia_kcal': 25, 'proteina_g': 3.6, 
                 'lipidios_g': 0.3, 'carboidratos_g': 4.0, 'fibra_g': 2.9, 'calcio_mg': 86, 
                 'vitamina_c_mg': 30},
    'Cebola': {'id': 107, 'umidade': 88.9, 'energia_kcal': 39, 'proteina_g': 1.7, 
               'lipidios_g': 0.1, 'carboidratos_g': 8.9, 'fibra_g': 2.2, 'calcio_mg': 14, 
               'vitamina_c_mg': 12},
    'Cenoura': {'id': 110, 'umidade': 90.1, 'energia_kcal': 34, 'proteina_g': 1.3, 
                'lipidios_g': 0.2, 'carboidratos_g': 7.7, 'fibra_g': 3.2, 'calcio_mg': 23, 
                'vitamina_c_mg': 11},
    'Chuchu': {'id': 113, 'umidade': 94.8, 'energia_kcal': 17, 'proteina_g': 0.7, 
               'lipidios_g': 0.1, 'carboidratos_g': 4.1, 'fibra_g': 1.3, 'calcio_mg': 12, 
               'vitamina_c_mg': 7},
    'Couve': {'id': 115, 'umidade': 90.9, 'energia_kcal': 27, 'proteina_g': 2.9, 
              'lipidios_g': 0.5, 'carboidratos_g': 4.3, 'fibra_g': 3.1, 'calcio_mg': 131, 
              'vitamina_c_mg': 35},
    'Goiaba': {'id': 197, 'umidade': 85.7, 'energia_kcal': 52, 'proteina_g': 0.9, 
              'lipidios_g': 0.5, 'carboidratos_g': 12.4, 'fibra_g': 6.3, 'calcio_mg': 5, 
              'vitamina_c_mg': 7},
   'Limão': {'id': 9004, 'umidade': 91.0, 'energia_kcal': 29, 'proteina_g': 1.1, 
             'lipidios_g': 0.3, 'carboidratos_g': 9.3, 'fibra_g': 2.8, 'calcio_mg': 26, 
             'vitamina_c_mg': 53},  # Limão Tahiti
   'Manga': {'id': 133, 'umidade': 83.5, 'energia_kcal': 65, 'proteina_g': 0.5, 
             'lipidios_g': 0.3, 'carboidratos_g': 15.0, 'fibra_g': 1.6, 'calcio_mg': 12, 
             'vitamina_c_mg': 43},
   'Maracujá': {'id': 232, 'umidade': 82.9, 'energia_kcal': 68, 'proteina_g': 2.0, 
                'lipidios_g': 2.1, 'carboidratos_g': 12.3, 'fibra_g': 1.1, 'calcio_mg': 5, 
                'vitamina_c_mg': 28},
   'Morango': {'id': 239, 'umidade': 91.5, 'energia_kcal': 30, 'proteina_g': 0.9, 
               'lipidios_g': 0.3, 'carboidratos_g': 6.8, 'fibra_g': 1.7, 'calcio_mg': 11, 
               'vitamina_c_mg': 10},
   'Pepino': {'id': 142, 'umidade': 96.8, 'energia_kcal': 10, 'proteina_g': 0.9, 
              'lipidios_g': 0.0, 'carboidratos_g': 2.0, 'fibra_g': 1.1, 'calcio_mg': 10, 
              'vitamina_c_mg': 9},
   'Pimentão': {'id': 144, 'umidade': 93.5, 'energia_kcal': 21, 'proteina_g': 1.1, 
                'lipidios_g': 0.2, 'carboidratos_g': 4.9, 'fibra_g': 2.6, 'calcio_mg': 9, 
                'vitamina_c_mg': 8},
   'Repolho': {'id': 150, 'umidade': 90.1, 'energia_kcal': 31, 'proteina_g': 1.9, 
               'lipidios_g': 0.1, 'carboidratos_g': 7.2, 'fibra_g': 2.0, 'calcio_mg': 44, 
               'vitamina_c_mg': 18},
   'Tangerina': {'id': 251, 'umidade': 89.2, 'energia_kcal': 38, 'proteina_g': 0.8, 
                 'lipidios_g': 0.1, 'carboidratos_g': 9.6, 'fibra_g': 0.9, 'calcio_mg': 13, 
                 'vitamina_c_mg': 8},
   'Tomate': {'id': 157, 'umidade': 95.1, 'energia_kcal': 15, 'proteina_g': 1.1, 
              'lipidios_g': 0.2, 'carboidratos_g': 3.1, 'fibra_g': 1.2, 'calcio_mg': 7, 
              'vitamina_c_mg': 11},
   'Banana': {'id': 182, 'umidade': 71.9, 'energia_kcal': 98, 'proteina_g': 1.3, 
              'lipidios_g': 0.1, 'carboidratos_g': 26.0, 'fibra_g': 2.0, 'calcio_mg': 8, 
              'vitamina_c_mg': 26},
   'Berinjela': {'id': 9006, 'umidade': 92.5, 'energia_kcal': 24, 'proteina_g': 1.0, 
                 'lipidios_g': 0.2, 'carboidratos_g': 5.7, 'fibra_g': 3.4, 'calcio_mg': 9, 
                 'vitamina_c_mg': 2},
   'Quiabo': {'id': 9007, 'umidade': 90.0, 'energia_kcal': 33, 'proteina_g': 2.0, 
              'lipidios_g': 0.1, 'carboidratos_g': 7.0, 'fibra_g': 3.2, 'calcio_mg': 81, 
              'vitamina_c_mg': 21},
   'Jiló': {'id': 9008, 'umidade': 92.0, 'energia_kcal': 27, 'proteina_g': 1.4, 
            'lipidios_g': 0.2, 'carboidratos_g': 6.2, 'fibra_g': 4.8, 'calcio_mg': 20, 
            'vitamina_c_mg': 6},
   'Gengibre': {'id': 9009, 'umidade': 78.9, 'energia_kcal': 80, 'proteina_g': 1.8, 
                'lipidios_g': 0.8, 'carboidratos_g': 17.8, 'fibra_g': 2.0, 'calcio_mg': 16, 
                'vitamina_c_mg': 5},
   'Uva': {'id': 9010, 'umidade': 80.5, 'energia_kcal': 69, 'proteina_g': 0.7, 
           'lipidios_g': 0.2, 'carboidratos_g': 17.1, 'fibra_g': 0.9, 'calcio_mg': 10, 
           'vitamina_c_mg': 4},
   'Lichia': {'id': 9011, 'umidade': 81.8, 'energia_kcal': 66, 'proteina_g': 0.8, 
              'lipidios_g': 0.4, 'carboidratos_g': 16.5, 'fibra_g': 1.3, 'calcio_mg': 5, 
              'vitamina_c_mg': 72},
   'Pitaia': {'id': 9012, 'umidade': 87.0, 'energia_kcal': 50, 'proteina_g': 1.1, 
              'lipidios_g': 0.4, 'carboidratos_g': 11.0, 'fibra_g': 3.0, 'calcio_mg': 9, 
              'vitamina_c_mg': 21},
   'Mamão': {'id': 9013, 'umidade': 88.6, 'energia_kcal': 43, 'proteina_g': 0.5, 
             'lipidios_g': 0.3, 'carboidratos_g': 10.8, 'fibra_g': 1.8, 'calcio_mg': 20, 
             'vitamina_c_mg': 62},
   'Coco': {'id': 9014, 'umidade': 47.0, 'energia_kcal': 354, 'proteina_g': 3.3, 
            'lipidios_g': 33.5, 'carboidratos_g': 15.2, 'fibra_g': 9.0, 'calcio_mg': 14, 
            'vitamina_c_mg': 3},
   'Graviola': {'id': 9015, 'umidade': 81.2, 'energia_kcal': 66, 'proteina_g': 1.0, 
                'lipidios_g': 0.3, 'carboidratos_g': 16.8, 'fibra_g': 3.3, 'calcio_mg': 14, 
                'vitamina_c_mg': 21},
   'Cajamanga': {'id': 9016, 'umidade': 91.0, 'energia_kcal': 31, 'proteina_g': 0.6, 
                 'lipidios_g': 0.1, 'carboidratos_g': 7.6, 'fibra_g': 0.9, 'calcio_mg': 3, 
                 'vitamina_c_mg': 34},
   'Atemóia': {'id': 9017, 'umidade': 72.9, 'energia_kcal': 94, 'proteina_g': 1.7, 
               'lipidios_g': 0.3, 'carboidratos_g': 23.6, 'fibra_g': 2.4, 'calcio_mg': 24, 
               'vitamina_c_mg': 36},
   'Mandioca': {'id': 9018, 'umidade': 61.8, 'energia_kcal': 151, 'proteina_g': 1.0, 
                'lipidios_g': 0.3, 'carboidratos_g': 36.2, 'fibra_g': 1.9, 'calcio_mg': 19, 
                'vitamina_c_mg': 17}
}

# Criar DataFrame de nutrientes
df_nutrientes = pd.DataFrame.from_dict(dados_taco, orient='index')

# Adicionar categorias aos alimentos
categorias_alimentos = {
   'folhosas': ['Alface', 'Couve', 'Agrião'],
   'frutos_hortalicas': ['Tomate', 'Pimentão', 'Berinjela', 'Jiló', 'Quiabo', 'Chuchu', 'Pepino'],
   'raizes_tuberculos': ['Batata', 'Mandioca', 'Beterraba', 'Cenoura'],
   'bulbos': ['Cebola'],
   'cruciferas': ['Brócolis', 'Repolho'],
   'frutas_doces': ['Banana', 'Manga', 'Mamão', 'Uva', 'Lichia', 'Atemóia'],
   'frutas_acidas': ['Limão', 'Tangerina', 'Maracujá', 'Cajamanga'],
   'frutas_berries': ['Morango'],
   'frutas_gordurosas': ['Abacate', 'Coco'],
   'frutas_tropicais': ['Goiaba', 'Graviola', 'Pitaia'],
   'temperos': ['Gengibre'],
   'leguminosas': ['Abóbora']
}

# Inverter o dicionário para facilitar a atribuição
categoria_por_alimento = {}
for categoria, alimentos in categorias_alimentos.items():
   for alimento in alimentos:
       categoria_por_alimento[alimento] = categoria

df_nutrientes['categoria'] = df_nutrientes.index.map(categoria_por_alimento)

# Calcular scores nutricionais normalizados
df_nutrientes['score_vitamina_c'] = df_nutrientes['vitamina_c_mg'] / df_nutrientes['vitamina_c_mg'].max()
df_nutrientes['score_fibras'] = df_nutrientes['fibra_g'] / df_nutrientes['fibra_g'].max()
df_nutrientes['score_baixa_caloria'] = 1 - (df_nutrientes['energia_kcal'] / df_nutrientes['energia_kcal'].max())
df_nutrientes['score_proteina'] = df_nutrientes['proteina_g'] / df_nutrientes['proteina_g'].max()

print("📊 Dados nutricionais carregados com sucesso!")
print(f"Total de alimentos com dados nutricionais: {len(df_nutrientes)}")
print("\n🥗 Categorias de alimentos:")
for categoria, count in df_nutrientes['categoria'].value_counts().items():
   print(f"  - {categoria}: {count} alimentos")
print("\n💊 Top 5 alimentos por vitamina C:")
print(df_nutrientes.nlargest(5, 'vitamina_c_mg')[['vitamina_c_mg', 'categoria']])

📊 Dados nutricionais carregados com sucesso!
Total de alimentos com dados nutricionais: 35

🥗 Categorias de alimentos:
  - frutos_hortalicas: 7 alimentos
  - frutas_doces: 6 alimentos
  - raizes_tuberculos: 4 alimentos
  - frutas_acidas: 4 alimentos
  - folhosas: 3 alimentos
  - frutas_tropicais: 3 alimentos
  - frutas_gordurosas: 2 alimentos
  - cruciferas: 2 alimentos
  - leguminosas: 1 alimentos
  - bulbos: 1 alimentos
  - frutas_berries: 1 alimentos
  - temperos: 1 alimentos

💊 Top 5 alimentos por vitamina C:
         vitamina_c_mg      categoria
Lichia              72   frutas_doces
Mamão               62   frutas_doces
Limão               53  frutas_acidas
Manga               43   frutas_doces
Atemóia             36   frutas_doces


In [53]:
"""
Dados reais de produção por região do DF
Fonte: Informações Agropecuárias do Distrito Federal - 2024 (EMATER-DF)
"""

# Dados de produção por escritório/região
producao_emater = {
    'ALEXANDRE GUSMÃO': {
        'Abacate': {'area_ha': 23.08, 'producao_t': 18.0, 'participacao_df': 23.08},
        'Alface': {'area_ha': 24.25, 'producao_t': 9.51, 'participacao_df': 24.25},
        'Banana': {'area_ha': 9.5, 'producao_t': 6.83, 'participacao_df': 9.5},
        'Chuchu': {'area_ha': 32.99, 'producao_t': 14.52, 'participacao_df': 32.99},
        'Couve': {'area_ha': 35.66, 'producao_t': 9.39, 'participacao_df': 35.66},
        'Goiaba': {'area_ha': 42.2, 'producao_t': 38.56, 'participacao_df': 42.2},
        'Limão': {'area_ha': 13.15, 'producao_t': 12.04, 'participacao_df': 13.15},
        'Mandioca': {'area_ha': 18.29, 'producao_t': 6.17, 'participacao_df': 18.29},
        'Manga': {'area_ha': 13.67, 'producao_t': 1.66, 'participacao_df': 13.67},
        'Maracujá': {'area_ha': 12.19, 'producao_t': 10.26, 'participacao_df': 12.19},
        'Pimentão': {'area_ha': 14.73, 'producao_t': 4.13, 'participacao_df': 14.73},
        'Repolho': {'area_ha': 24.68, 'producao_t': 8.87, 'participacao_df': 24.68},
        'Tangerina': {'area_ha': 21.44, 'producao_t': 6.96, 'participacao_df': 21.44},
        'Tomate': {'area_ha': 14.62, 'producao_t': 11.67, 'participacao_df': 14.62},
        'Uva': {'area_ha': 4.16, 'producao_t': 0.77, 'participacao_df': 4.16}
    },
    'BRAZLÂNDIA': {
        'Abacate': {'area_ha': 19.45, 'producao_t': 18.6, 'participacao_df': 19.45},
        'Alface': {'area_ha': 8.87, 'producao_t': 5.95, 'participacao_df': 8.87},
        'Banana': {'area_ha': 4.35, 'producao_t': 3.61, 'participacao_df': 4.35},
        'Chuchu': {'area_ha': 27.72, 'producao_t': 13.09, 'participacao_df': 27.72},
        'Couve': {'area_ha': 9.43, 'producao_t': 3.24, 'participacao_df': 9.43},
        'Goiaba': {'area_ha': 53.62, 'producao_t': 57.58, 'participacao_df': 53.62},
        'Limão': {'area_ha': 9.3, 'producao_t': 9.48, 'participacao_df': 9.3},
        'Mandioca': {'area_ha': 6.88, 'producao_t': 4.08, 'participacao_df': 6.88},
        'Manga': {'area_ha': 4.4, 'producao_t': 0.6, 'participacao_df': 4.4},
        'Maracujá': {'area_ha': 3.94, 'producao_t': 1.38, 'participacao_df': 3.94},
        'Morango': {'area_ha': 76.51, 'producao_t': 17.36, 'participacao_df': 76.51},
        'Pimentão': {'area_ha': 15.89, 'producao_t': 6.18, 'participacao_df': 15.89},
        'Repolho': {'area_ha': 23.53, 'producao_t': 10.6, 'participacao_df': 23.53},
        'Tangerina': {'area_ha': 11.42, 'producao_t': 5.63, 'participacao_df': 11.42},
        'Tomate': {'area_ha': 8.66, 'producao_t': 10.33, 'participacao_df': 8.66},
        'Uva': {'area_ha': 3.29, 'producao_t': 0.24, 'participacao_df': 3.29}
    },
    'CEILÂNDIA': {
        'Abacate': {'area_ha': 3.61, 'producao_t': 9.77, 'participacao_df': 3.61},
        'Alface': {'area_ha': 24.1, 'producao_t': 12.04, 'participacao_df': 24.1},
        'Banana': {'area_ha': 12.33, 'producao_t': 29.03, 'participacao_df': 12.33},
        'Chuchu': {'area_ha': 29.16, 'producao_t': 17.64, 'participacao_df': 29.16},
        'Couve': {'area_ha': 22.59, 'producao_t': 7.41, 'participacao_df': 22.59},
        'Goiaba': {'area_ha': 0.37, 'producao_t': 1.44, 'participacao_df': 0.37},
        'Limão': {'area_ha': 10.43, 'producao_t': 23.69, 'participacao_df': 10.43},
        'Mandioca': {'area_ha': 16.23, 'producao_t': 9.48, 'participacao_df': 16.23},
        'Manga': {'area_ha': 5.92, 'producao_t': 3.63, 'participacao_df': 5.92},
        'Maracujá': {'area_ha': 7.96, 'producao_t': 20.18, 'participacao_df': 7.96},
        'Repolho': {'area_ha': 17.83, 'producao_t': 8.75, 'participacao_df': 17.83},
        'Tangerina': {'area_ha': 6.03, 'producao_t': 9.55, 'participacao_df': 6.03},
        'Tomate': {'area_ha': 3.22, 'producao_t': 3.47, 'participacao_df': 3.22}
    },
    'GAMA': {
        'Abacate': {'area_ha': 6.98, 'producao_t': 20.49, 'participacao_df': 6.98},
        'Alface': {'area_ha': 19.58, 'producao_t': 32.92, 'participacao_df': 19.58},
        'Banana': {'area_ha': 5.59, 'producao_t': 15.58, 'participacao_df': 5.59},
        'Couve': {'area_ha': 9.74, 'producao_t': 8.83, 'participacao_df': 9.74},
        'Lichia': {'area_ha': 5.86, 'producao_t': 1.21, 'participacao_df': 5.86},
        'Limão': {'area_ha': 12.17, 'producao_t': 27.56, 'participacao_df': 12.17},
        'Mandioca': {'area_ha': 9.66, 'producao_t': 16.37, 'participacao_df': 9.66},
        'Manga': {'area_ha': 13.1, 'producao_t': 6.94, 'participacao_df': 13.1},
        'Maracujá': {'area_ha': 3.06, 'producao_t': 7.6, 'participacao_df': 3.06},
        'Pimentão': {'area_ha': 2.24, 'producao_t': 3.26, 'participacao_df': 2.24},
        'Pitaia': {'area_ha': 16.31, 'producao_t': 1.71, 'participacao_df': 16.31},
        'Repolho': {'area_ha': 1.85, 'producao_t': 2.31, 'participacao_df': 1.85},
        'Tangerina': {'area_ha': 9.07, 'producao_t': 10.7, 'participacao_df': 9.07}
    },
    'PLANALTINA': {
        'Abacate': {'area_ha': 4.26, 'producao_t': 8.23, 'participacao_df': 4.26},
        'Alface': {'area_ha': 1.13, 'producao_t': 3.94, 'participacao_df': 1.13},
        'Banana': {'area_ha': 7.35, 'producao_t': 10.82, 'participacao_df': 7.35},
        'Chuchu': {'area_ha': 1.94, 'producao_t': 4.52, 'participacao_df': 1.94},
        'Couve': {'area_ha': 0.96, 'producao_t': 1.69, 'participacao_df': 0.96},
        'Goiaba': {'area_ha': 0.8, 'producao_t': 2.57, 'participacao_df': 0.8},
        'Lichia': {'area_ha': 61.0, 'producao_t': 7.17, 'participacao_df': 61.0},
        'Limão': {'area_ha': 15.68, 'producao_t': 27.24, 'participacao_df': 15.68},
        'Mandioca': {'area_ha': 7.19, 'producao_t': 30.89, 'participacao_df': 7.19},
        'Manga': {'area_ha': 13.01, 'producao_t': 3.23, 'participacao_df': 13.01},
        'Maracujá': {'area_ha': 7.53, 'producao_t': 11.02, 'participacao_df': 7.53},
        'Pimentão': {'area_ha': 2.1, 'producao_t': 2.39, 'participacao_df': 2.1},
        'Repolho': {'area_ha': 1.68, 'producao_t': 3.74, 'participacao_df': 1.68},
        'Tangerina': {'area_ha': 10.55, 'producao_t': 8.73, 'participacao_df': 10.55},
        'Tomate': {'area_ha': 3.49, 'producao_t': 16.83, 'participacao_df': 3.49},
        'Uva': {'area_ha': 15.24, 'producao_t': 11.13, 'participacao_df': 15.24}
    },
    'SOBRADINHO': {
        'Abacate': {'area_ha': 5.78, 'producao_t': 11.67, 'participacao_df': 5.78},
        'Alface': {'area_ha': 11.79, 'producao_t': 12.59, 'participacao_df': 11.79},
        'Banana': {'area_ha': 27.36, 'producao_t': 43.04, 'participacao_df': 27.36},
        'Berinjela': {'area_ha': 14.33, 'producao_t': 4.35, 'participacao_df': 14.33},
        'Chuchu': {'area_ha': 2.32, 'producao_t': 3.22, 'participacao_df': 2.32},
        'Couve': {'area_ha': 9.15, 'producao_t': 5.88, 'participacao_df': 9.15},
        'Limão': {'area_ha': 7.6, 'producao_t': 11.29, 'participacao_df': 7.6},
        'Mandioca': {'area_ha': 15.39, 'producao_t': 14.95, 'participacao_df': 15.39},
        'Manga': {'area_ha': 25.42, 'producao_t': 10.15, 'participacao_df': 25.42},
        'Maracujá': {'area_ha': 8.8, 'producao_t': 3.45, 'participacao_df': 8.8},
        'Pimentão': {'area_ha': 5.13, 'producao_t': 6.43, 'participacao_df': 5.13},
        'Repolho': {'area_ha': 11.8, 'producao_t': 12.24, 'participacao_df': 11.8},
        'Tangerina': {'area_ha': 7.6, 'producao_t': 6.44, 'participacao_df': 7.6},
        'Tomate': {'area_ha': 4.5, 'producao_t': 8.75, 'participacao_df': 4.5},
        'Uva': {'area_ha': 22.17, 'producao_t': 5.35, 'participacao_df': 22.17}
    },
    'SÃO SEBASTIÃO': {
        'Abacate': {'area_ha': 0.1, 'producao_t': 1.86, 'participacao_df': 0.1},
        'Abóbora': {'area_ha': 4.14, 'producao_t': 0.19, 'participacao_df': 4.14},
        'Agrião': {'area_ha': 0.94, 'producao_t': 0.1, 'participacao_df': 0.94},
        'Alface': {'area_ha': 0.46, 'producao_t': 0.34, 'participacao_df': 0.46},
        'Banana': {'area_ha': 1.5, 'producao_t': 28.78, 'participacao_df': 1.5},
        'Cebola': {'area_ha': 66.0, 'producao_t': 95.49, 'participacao_df': 66.0},
        'Couve': {'area_ha': 0.59, 'producao_t': 0.22, 'participacao_df': 0.59},
        'Limão': {'area_ha': 1.05, 'producao_t': 11.77, 'participacao_df': 1.05},
        'Mandioca': {'area_ha': 2.6, 'producao_t': 1.98, 'participacao_df': 2.6},
        'Manga': {'area_ha': 5.63, 'producao_t': 20.2, 'participacao_df': 5.63},
        'Maracujá': {'area_ha': 8.45, 'producao_t': 28.31, 'participacao_df': 8.45},
        'Morango': {'area_ha': 2.41, 'producao_t': 0.46, 'participacao_df': 2.41},
        'Pitaia': {'area_ha': 0.37, 'producao_t': 0.31, 'participacao_df': 0.37},
        'Quiabo': {'area_ha': 1.16, 'producao_t': 0.09, 'participacao_df': 1.16},
        'Repolho': {'area_ha': 0.1, 'producao_t': 0.09, 'participacao_df': 0.1},
        'Tangerina': {'area_ha': 0.27, 'producao_t': 1.36, 'participacao_df': 0.27},
        'Tomate': {'area_ha': 0.17, 'producao_t': 0.18, 'participacao_df': 0.17}
    }
}

# Converter para DataFrame
lista_producao = []
for regiao, produtos_dict in producao_emater.items(): # Renomeado para produtos_dict para clareza
    for produto, dados in produtos_dict.items():
        lista_producao.append({
            'regiao': regiao,
            'produto': produto,
            'area_ha': dados['area_ha'],
            'producao_t': dados['producao_t']
        })

In [54]:
df_producao = pd.DataFrame(lista_producao)

# Mapear os nomes dos produtos da EMATER para os nomes do nosso escopo

df_producao['produto'] = df_producao['produto'].replace(mapeamento_produtos)
df_producao['produto'] = df_producao['produto'].str.strip()

# Filtrar apenas produtos que estão no nosso escopo
df_producao = df_producao[df_producao['produto'].isin(produtos_escopo)].copy()

# Calcular a produção total por produto no DF para ponderação
df_producao_total_produto = df_producao.groupby('produto', as_index=False)['producao_t'].sum()
df_producao_total_produto.rename(columns={'producao_t': 'producao_total_df_t'}, inplace=True)

# Adicionar a produção total ao df_producao
df_producao = pd.merge(df_producao, df_producao_total_produto, on='produto', how='left')

# Calcular a relevância da região para cada produto (porcentagem da produção do DF naquela região)
df_producao['relevancia_regiao_percent'] = 0.0
mask_total_prod_gt_0 = df_producao['producao_total_df_t'] > 0
df_producao.loc[mask_total_prod_gt_0, 'relevancia_regiao_percent'] = \
    (df_producao['producao_t'][mask_total_prod_gt_0] / df_producao['producao_total_df_t'][mask_total_prod_gt_0]) * 100
df_producao['relevancia_regiao_percent'].fillna(0, inplace=True)

 Análise dos dados EMATER

In [55]:
print("\n\nDados de produção da EMATER-DF processados (com dados do usuário)!")
print(f"Total de registros de produção regional (após filtro de escopo): {len(df_producao)}")
print(f"Total de produtos distintos na EMATER (no escopo): {df_producao['produto'].nunique()}")

print("\nTop regiões produtoras de Alface (por participação %):")
alface_prod = df_producao[df_producao['produto'] == 'Alface'].nlargest(5, 'relevancia_regiao_percent')
print(alface_prod[['regiao', 'producao_t', 'relevancia_regiao_percent']])

print("\nTop produtos mais produzidos no DF (toneladas):")
top_5_produtos = df_producao.groupby('produto')['producao_t'].sum().nlargest()
print(top_5_produtos)

print("\nExemplo de dados de produção para a região de BRAZLÂNDIA (ordenado por produção):")
brazlandia_data = df_producao[df_producao['regiao'] == 'BRAZLÂNDIA'].sort_values(by='producao_t', ascending=False)
print(brazlandia_data[['produto', 'area_ha', 'producao_t', 'relevancia_regiao_percent']].head())

df_producao.head()



Dados de produção da EMATER-DF processados (com dados do usuário)!
Total de registros de produção regional (após filtro de escopo): 105
Total de produtos distintos na EMATER (no escopo): 23

Top regiões produtoras de Alface (por participação %):
              regiao  producao_t  relevancia_regiao_percent
45              GAMA       32.92                  42.592832
74        SOBRADINHO       12.59                  16.289300
32         CEILÂNDIA       12.04                  15.577694
1   ALEXANDRE GUSMÃO        9.51                  12.304308
16        BRAZLÂNDIA        5.95                   7.698279

Top produtos mais produzidos no DF (toneladas):
produto
Banana     137.69
Limão      123.07
Goiaba     100.15
Cebola      95.49
Abacate     88.62
Name: producao_t, dtype: float64

Exemplo de dados de produção para a região de BRAZLÂNDIA (ordenado por produção):
    produto  area_ha  producao_t  relevancia_regiao_percent
20   Goiaba    53.62       57.58                  57.493759
15  Abaca

,regiao,produto,area_ha,producao_t,producao_total_df_t,relevancia_regiao_percent
0,ALEXANDRE GUSMÃO,Abacate,23.08,18.00,88.62,20.311442
1,ALEXANDRE GUSMÃO,Alface,24.25,9.51,77.29,12.304308
2,ALEXANDRE GUSMÃO,Banana,9.50,6.83,137.69,4.960418
3,ALEXANDRE GUSMÃO,Chuchu,32.99,14.52,52.99,27.401396
4,ALEXANDRE GUSMÃO,Couve,35.66,9.39,36.66,25.613748


In [56]:
"""
Criar uma matriz de utilidade Usuário-Item (Consumidor-Associação).
Esta matriz simula avaliações de consumidores para as associações.
O objetivo é ter pelo menos 5.000 avaliações (linhas na forma longa).
"""
np.random.seed(101)

num_consumidores_simulados = 500
min_avaliacoes_por_consumidor = 10
max_avaliacoes_por_consumidor = 25

consumidores_ids = [f'Consumidor_{i+1:03d}' for i in range(num_consumidores_simulados)]

if 'df_associacoes' not in locals() or df_associacoes.empty:
    print("⚠️ df_associacoes não definido ou vazio. Célula 2 precisa ser executada.")
    df_associacoes = pd.DataFrame({'id': range(1,18), 'avaliacao_media': np.random.uniform(3.0,5.0,17), 'produtos': [[] for _ in range(17)]})


associacoes_ids_avaliadas = df_associacoes['id'].unique()
if len(associacoes_ids_avaliadas) == 0:
    raise ValueError("Nenhuma ID de associação encontrada. Verifique o df_associacoes.")

utility_matrix_list = []
for consumidor_id in consumidores_ids:
    num_avaliacoes_a_fazer = np.random.randint(min_avaliacoes_por_consumidor,
                                               min(max_avaliacoes_por_consumidor, len(associacoes_ids_avaliadas)) + 1)
    if num_avaliacoes_a_fazer == 0 and len(associacoes_ids_avaliadas) > 0 : # Garante pelo menos 1 se possível
        num_avaliacoes_a_fazer = 1
    if num_avaliacoes_a_fazer == 0 : continue # Pula se não há associações para avaliar

    associacoes_escolhidas_ids = np.random.choice(associacoes_ids_avaliadas, size=num_avaliacoes_a_fazer, replace=False)

    for assoc_id in associacoes_escolhidas_ids:
        assoc_info = df_associacoes[df_associacoes['id'] == assoc_id].iloc[0]
        avaliacao_base = assoc_info['avaliacao_media']
        if np.random.rand() < 0.6: # 60% de chance da avaliação ser próxima da média
            avaliacao = np.clip(round(np.random.normal(loc=avaliacao_base, scale=0.5)), 1, 5)
        else: # 40% de chance de uma avaliação mais aleatória
            avaliacao = np.random.randint(1, 6)
        utility_matrix_list.append({
            'id_consumidor': consumidor_id,
            'id_associacao': assoc_id,
            'avaliacao': int(avaliacao)
        })

df_utility_long = pd.DataFrame(utility_matrix_list)
print(f"✅ Matriz de utilidade em formato longo criada com {len(df_utility_long)} avaliações.")


if not df_utility_long.empty:
    df_utility_pivot = df_utility_long.pivot(
        index='id_consumidor',
        columns='id_associacao',
        values='avaliacao'
    )
    print("\nMatriz de Utilidade Pivotada (amostra - NaN significa sem avaliação):")
    print(df_utility_pivot.head())
    print(f"\nDimensões da matriz pivotada: {df_utility_pivot.shape}")
    print(f"Esparsidade da matriz: {1.0 - (df_utility_pivot.count().sum() / float(df_utility_pivot.shape[0] * df_utility_pivot.shape[1])):.4f}")
else:
    print("Matriz de utilidade longa está vazia. Pivot não pode ser criado.")
    # Criar um df_utility_pivot vazio para evitar erros subsequentes, mas o sistema não será funcional
    df_utility_pivot = pd.DataFrame()

✅ Matriz de utilidade em formato longo criada com 6714 avaliações.

Matriz de Utilidade Pivotada (amostra - NaN significa sem avaliação):
id_associacao    1    2    3    4    5    6    7    8    9    10   11   12   13   14   15   16   17
id_consumidor                                                                                      
Consumidor_001  3.0  4.0  2.0  5.0  2.0  5.0  4.0  4.0  5.0  4.0  5.0  3.0  2.0  4.0  4.0  4.0  5.0
Consumidor_002  5.0  5.0  4.0  4.0  NaN  4.0  4.0  NaN  NaN  5.0  3.0  NaN  5.0  5.0  5.0  4.0  4.0
Consumidor_003  4.0  5.0  4.0  4.0  NaN  4.0  5.0  2.0  5.0  3.0  4.0  5.0  4.0  5.0  5.0  3.0  3.0
Consumidor_004  NaN  1.0  NaN  1.0  5.0  3.0  4.0  1.0  NaN  NaN  4.0  5.0  4.0  5.0  NaN  5.0  NaN
Consumidor_005  4.0  NaN  4.0  2.0  4.0  NaN  4.0  3.0  5.0  4.0  4.0  4.0  NaN  5.0  4.0  5.0  2.0

Dimensões da matriz pivotada: (500, 17)
Esparsidade da matriz: 0.2101


## Classe `SistemaRecomendacaoDF`

Esta classe é o componente central do sistema, encapsulando toda a lógica para gerar recomendações híbridas de associações/cooperativas agrícolas para os consumidores. Ela combina informações baseadas em conteúdo (características das associações e produtos, localização) com uma abordagem de filtragem colaborativa item-item.

### 1. Inicialização (`__init__`)

O construtor da classe é responsável por receber os DataFrames essenciais e preparar estruturas internas necessárias para o processo de recomendação.

**Parâmetros de Entrada:**

* `df_associacoes` (pd.DataFrame): Um DataFrame contendo os dados cadastrais das associações/cooperativas. Deve incluir colunas como `id`, `nome`, `latitude`, `longitude`, `produtos` (lista dos produtos oferecidos), `organico_principal` (booleano), `avaliacao_media`, etc.
* `df_nutrientes` (pd.DataFrame): Um DataFrame com informações nutricionais dos produtos. O índice deve ser o nome do produto, e as colunas devem incluir os scores nutricionais relevantes (ex: `score_vitamina_c`, `score_fibras`, `score_baixa_caloria`).
* `df_producao` (pd.DataFrame): Um DataFrame contendo dados de produção regional (ex: da EMATER-DF). Deve incluir colunas como `regiao`, `produto`, e `relevancia_regiao_percent`.
* `df_utility_pivot` (pd.DataFrame): A matriz de utilidade no formato pivotado, onde o índice são os `id_consumidor`, as colunas são os `id_associacao`, e os valores são as `avaliacao` dadas.

**Principais Ações Realizadas na Inicialização:**

1.  **Armazenamento dos DataFrames**: Cópias dos DataFrames fornecidos são armazenadas como atributos da instância (ex: `self.df_associacoes = df_associacoes.copy()`).
2.  **Preparação da Matriz de Utilidade Preenchida**:
    * `self.utility_matrix_filled = self.df_utility_pivot.fillna(0)`: Uma versão da matriz de utilidade é criada onde todos os valores `NaN` (indicando que um consumidor não avaliou uma associação) são substituídos por `0`. Isso é crucial para os cálculos de similaridade.
3.  **Cálculo da Matriz de Similaridade Item-Item (`self.item_similarity_df`)**:
    * Esta matriz armazena o quão "similares" duas associações (itens) são, com base nos padrões de como foram avaliadas pelos mesmos consumidores.
    * **Condição**: É calculada apenas se `self.utility_matrix_filled` não estiver vazia e tiver mais de uma associação (coluna).
    * **Método**: É utilizada a função `cosine_similarity` da biblioteca `sklearn.metrics.pairwise` sobre a transposta da `self.utility_matrix_filled` (`self.utility_matrix_filled.T`). A transposição é feita porque a similaridade do cosseno é calculada entre as linhas (que, após a transposição, representam as associações).
    * **Resultado**: `self.item_similarity_df` é um DataFrame onde tanto o índice quanto as colunas são os `id_associacao`, e os valores representam a similaridade do cosseno entre cada par de associações.
    * Uma mensagem é impressa indicando se o cálculo foi bem-sucedido e as dimensões da matriz de similaridade.

---

### 2. Métodos Auxiliares (Privados)

A classe utiliza métodos privados (prefixados com `_`) para encapsular funcionalidades específicas:

* **`_calcular_distancia_km(self, lat1, lon1, lat2, lon2)`**
    * **Propósito**: Calcula a distância geodésica (a mais curta sobre a superfície de um elipsoide) em quilômetros entre dois pontos geográficos definidos por suas latitudes e longitudes.
    * **Parâmetros**: `lat1`, `lon1` (latitude/longitude do primeiro ponto), `lat2`, `lon2` (latitude/longitude do segundo ponto).
    * **Retorno**: A distância em km. Retorna `float('inf')` se alguma coordenada for inválida (NaN).
    * **Utilização Interna**: Usado pelo método `recomendar` para calcular a distância entre a localização do usuário e cada associação.

* **`_get_item_collab_score(self, id_consumidor, id_item_candidato)`**
    * **Propósito**: Estima um score de preferência para uma `id_item_candidato` (associação candidata) com base no histórico de avaliações do `id_consumidor` e na similaridade da associação candidata com outras associações que o consumidor já avaliou bem. Esta é a essência da filtragem colaborativa item-item.
    * **Parâmetros**:
        * `id_consumidor` (str): O ID do consumidor.
        * `id_item_candidato` (int/str): O ID da associação para a qual o score está sendo calculado.
    * **Funcionamento**:
        1.  Retorna `0.0` se o consumidor não existir na `self.utility_matrix_filled`, se o item candidato não estiver na `self.item_similarity_df`, ou se a matriz de similaridade estiver vazia.
        2.  Obtém todas as avaliações (`avaliacoes_consumidor`) que o `id_consumidor` deu.
        3.  Filtra para obter apenas os `itens_avaliados_bem` (aqueles com avaliação `>= 3.5`).
        4.  Itera sobre cada `id_item_gostou` (associação bem avaliada):
            * Se o `id_item_gostou` existir na matriz de similaridade e for diferente do `id_item_candidato`:
                * Busca a `similaridade` entre o `id_item_candidato` e o `id_item_gostou` em `self.item_similarity_df`.
                * Multiplica essa `similaridade` pela `avaliacao` que o consumidor deu ao `id_item_gostou`.
                * Acumula esse valor ponderado em `score_collab` e incrementa `num_itens_sim_usados`.
        5.  O score final é a média dos scores ponderados (`score_collab / num_itens_sim_usados`), ou `0.0` se nenhum item similar bem avaliado foi usado.
    * **Retorno**: Um float representando o score colaborativo calculado.

---

### 3. Método Principal de Recomendação (`recomendar`)

Este é o método público principal da classe, usado para gerar a lista final de associações recomendadas para um usuário, dadas suas preferências e localização.

**Parâmetros de Entrada:**

* `id_consumidor` (str): O identificador do consumidor.
* `lat_usuario` (float): A latitude da localização atual do usuário.
* `lon_usuario` (float): A longitude da localização atual do usuário.
* `preferencias` (dict): Um dicionário contendo as diversas preferências e pesos que guiarão a recomendação. (Consulte o guia de parâmetros de `user_preferences` para detalhes sobre as chaves e valores esperados).

**Principais Passos do Processo de Recomendação:**

1.  **Preparação Inicial (`cand_df`)**:
    * Uma cópia do `self.df_associacoes` é criada como o DataFrame de candidatos (`cand_df`).
    * A coluna `dist_km` é calculada para cada associação em `cand_df` em relação à `lat_usuario` e `lon_usuario`, utilizando `_calcular_distancia_km`.

2.  **Filtragem de Candidatos**:
    * **Distância**: As associações são filtradas para manter apenas aquelas dentro da `distancia_max_km` especificada nas `preferencias`.
    * **Orgânicos**: Se `apenas_organicos` for `True` nas `preferencias`, somente associações com `organico_principal == True` são mantidas.
    * **Produtos Desejados**: Se uma lista de `produtos_desejados` é fornecida nas `preferencias`, o `cand_df` é filtrado para manter apenas as associações que oferecem pelo menos um dos produtos listados.
    * Se `cand_df` ficar vazio após estas filtragens, uma mensagem é impressa e um DataFrame vazio é retornado.

3.  **Cálculo de Sub-Scores (Scores Normalizados)**:
    Para cada associação restante em `cand_df`, os seguintes scores são calculados e adicionados como novas colunas. Estes scores são geralmente normalizados ou concebidos para estarem numa escala comparável (frequentemente 0-1).
    * **`s_dist` (Score de Distância)**: Valoriza a proximidade. É calculado como `(1 - (dist_km - min_dist_encontrada) / (max_dist_encontrada - min_dist_encontrada))`. Se todas as distâncias forem iguais, o score é `1.0`.
    * **`s_aval` (Score de Avaliação)**: Baseado na `avaliacao_media` da associação, normalizada por 5.0 (assumindo que as avaliações vão até 5).
    * **`s_nutri` (Score Nutricional)**: Se um `objetivo_nutricional` e `produtos_desejados` são especificados, este score é a média dos scores nutricionais (ex: `score_vitamina_c`) dos produtos desejados que a associação oferece.
    * **`s_relev_prod` (Score de Relevância Produtiva Regional)**: Se `considerar_relevancia_produtiva_regiao` for `True` e `produtos_desejados` forem especificados, este score é a média da `relevancia_regiao_percent` (normalizada para 0-1) das regiões da associação para os produtos desejados.
    * **`s_collab` (Score Colaborativo)**: Calculado chamando `_get_item_collab_score`. O resultado é então normalizado dividindo-se pelo valor máximo de `s_collab` encontrado entre as candidatas (se este máximo for maior que zero), para que fique na escala de 0 a 1.

4.  **Cálculo do `score_final`**:
    * Uma coluna `score_final` é inicializada com `0.0`.
    * Cada sub-score (`s_dist`, `s_aval`, etc.) é multiplicado pelo seu respectivo peso (obtido do dicionário `preferencias`, ex: `peso_distancia`) e adicionado ao `score_final`.
    * `score_final = (s_dist * peso_distancia) + (s_aval * peso_avaliacao) + ...`

5.  **Ranking e Seleção das Recomendações Finais**:
    * O `cand_df` é ordenado em ordem decrescente pelo `score_final`.
    * As `top_n` primeiras associações (onde `top_n` é definido nas `preferencias`) são selecionadas para formar o DataFrame `rec_finais`.

**Retorno:**

* Um DataFrame Pandas (`rec_finais`) contendo as associações recomendadas.
* **Colunas Incluídas no DataFrame de Retorno**: `id`, `nome`, `dist_km` (distância real), `avaliacao_media` (original), `produtos` (lista de produtos da associação), `score_final` (o score agregado), e todos os sub-scores calculados: `s_dist`, `s_aval`, `s_nutri`, `s_relev_prod`, `s_collab`.

---

In [67]:
"""
Esta classe encapsulará toda a lógica para gerar recomendações híbridas.
"""
class SistemaRecomendacaoDF:
    def __init__(self, associations_data_df, nutritional_info_df, regional_production_df, consumer_ratings_pivot_df):
        self.associations_df = associations_data_df.copy()
        self.nutritional_info_df = nutritional_info_df.copy()
        self.regional_production_df = regional_production_df.copy()
        self.consumer_ratings_pivot_df = consumer_ratings_pivot_df.copy()
        
        # Preenche NaNs (ausência de avaliação) com 0 para cálculos de similaridade
        self.ratings_matrix_filled = self.consumer_ratings_pivot_df.fillna(0)

        self.association_similarity_df = pd.DataFrame() # Matriz de similaridade Associação-Associação
        if not self.ratings_matrix_filled.empty and self.ratings_matrix_filled.shape[1] > 1:
            # Calcula a similaridade do cosseno entre associações (itens)
            # Transpõe a matriz para que as associações sejam as linhas
            item_similarity_matrix = cosine_similarity(self.ratings_matrix_filled.T)
            self.association_similarity_df = pd.DataFrame(
                item_similarity_matrix,
                index=self.ratings_matrix_filled.columns, # IDs das associações
                columns=self.ratings_matrix_filled.columns # IDs das associações
            )
            print(f"Matriz de similaridade Associação-Associação calculada ({self.association_similarity_df.shape}).")
        else:
            print("Matriz de utilidade vazia ou com poucos itens. Similaridade Associação-Associação não calculada.")
        print("SistemaRecomendacaoDF inicializado.")

    def _calculate_distance_km(self, lat1, lon1, lat2, lon2):
        """Calcula a distância geodésica entre dois pontos em km."""
        if pd.isna(lat1) or pd.isna(lon1) or pd.isna(lat2) or pd.isna(lon2):
            return float('inf') # Retorna infinito se alguma coordenada for inválida
        return geodesic((lat1, lon1), (lat2, lon2)).km

    def _get_association_collaborative_score(self, consumer_id, candidate_association_id):
        """Calcula o score colaborativo para uma associação candidata com base nas avaliações do consumidor."""
        collaborative_score = 0.0
        num_similar_items_used = 0

        # Verifica se temos dados para o consumidor e para a associação candidata
        if consumer_id not in self.ratings_matrix_filled.index or \
           candidate_association_id not in self.association_similarity_df.index or \
           self.association_similarity_df.empty:
            return 0.0

        consumer_ratings_for_all_associations = self.ratings_matrix_filled.loc[consumer_id]
        # Considera apenas associações que o consumidor avaliou bem (ex: >= 3.5 estrelas)
        well_rated_association_ids = consumer_ratings_for_all_associations[consumer_ratings_for_all_associations >= 3.5].index

        for liked_association_id in well_rated_association_ids:
            # Verifica se a associação apreciada está na matriz de similaridade e não é a própria candidata
            if liked_association_id in self.association_similarity_df.columns and liked_association_id != candidate_association_id:
                similarity_value = self.association_similarity_df.loc[candidate_association_id, liked_association_id]
                collaborative_score += similarity_value * consumer_ratings_for_all_associations[liked_association_id]
                num_similar_items_used += 1
        
        # Média ponderada dos scores de similaridade
        return (collaborative_score / num_similar_items_used) if num_similar_items_used > 0 else 0.0

    def recomendar(self, consumer_id, user_latitude, user_longitude, user_preferences):
        """
        Gera recomendações de associações com base nas preferências do usuário e filtros.
        """
        print(f"\nGerando recomendações para '{consumer_id}' em ({user_latitude:.4f}, {user_longitude:.4f})")
        print(f"Preferências: {user_preferences.get('desired_products', 'N/A')}, Dist. Max: {user_preferences.get('max_distance_km', 'N/A')}km")

        candidate_associations_df = self.associations_df.copy()

        # 1. Filtro de Distância
        candidate_associations_df['distance_km'] = candidate_associations_df.apply(
            lambda row: self._calculate_distance_km(user_latitude, user_longitude, row['latitude'], row['longitude']), axis=1
        )
        max_dist_km_pref = user_preferences.get('max_distance_km', 30) # Padrão de 30km
        candidate_associations_df = candidate_associations_df[candidate_associations_df['distance_km'] <= max_dist_km_pref].copy()

        # 2. Filtro de Orgânicos
        if user_preferences.get('only_organic', False):
            candidate_associations_df = candidate_associations_df[candidate_associations_df['organico_principal'] == True].copy()

        # 3. Filtro de Produtos Desejados
        desired_products_list = user_preferences.get('desired_products', [])
        if desired_products_list: # Se a lista não estiver vazia
            candidate_associations_df = candidate_associations_df[
                candidate_associations_df['produtos'].apply(
                    lambda offered_products: any(desired_product in offered_products for desired_product in desired_products_list)
                )
            ].copy()

        if candidate_associations_df.empty:
            print("Nenhuma associação candidata encontrada após filtros iniciais.")
            return pd.DataFrame()
        print(f"{len(candidate_associations_df)} associações candidatas após filtros iniciais.")

        # --- Cálculo dos Scores para Ranking ---
        candidate_associations_df['final_score'] = 0.0 # Inicializa o score final

        # Score de Distância (normalizado: 0 a 1, onde 1 é melhor/mais perto)
        max_dist_found = candidate_associations_df['distance_km'].max()
        min_dist_found = candidate_associations_df['distance_km'].min()
        if max_dist_found > min_dist_found: # Evita divisão por zero se todas as distâncias forem iguais
            candidate_associations_df['normalized_distance_score'] = (1 - (candidate_associations_df['distance_km'] - min_dist_found) / (max_dist_found - min_dist_found))
        else:
            candidate_associations_df['normalized_distance_score'] = 1.0 # Todas têm a "melhor" distância (ou única)
        candidate_associations_df['final_score'] += candidate_associations_df['normalized_distance_score'] * user_preferences.get('weight_distance', 0.25)

        # Score de Avaliação da Associação (normalizado: 0 a 1)
        candidate_associations_df['normalized_rating_score'] = candidate_associations_df['avaliacao_media'] / 5.0
        candidate_associations_df['final_score'] += candidate_associations_df['normalized_rating_score'] * user_preferences.get('weight_rating', 0.20)

        # Score Nutricional
        nutritional_goal_pref = user_preferences.get('nutritional_goal')
        candidate_associations_df['normalized_nutritional_score'] = 0.0
        if nutritional_goal_pref and desired_products_list:
            nutritional_column_map = {
                'alta_vitamina_c': 'score_vitamina_c',
                'alta_fibra': 'score_fibras',
                'baixa_caloria': 'score_baixa_caloria'
                # Adicionar outros scores nutricionais aqui se definidos no df_nutrientes
            }
            score_column_name = nutritional_column_map.get(nutritional_goal_pref)
            if score_column_name and score_column_name in self.nutritional_info_df.columns:
                for index, row in candidate_associations_df.iterrows():
                    association_nutritional_score_sum = 0
                    nutritional_product_count = 0
                    # Considera apenas os produtos desejados que a associação oferece
                    relevant_products_for_nutrition = [p for p in row['produtos'] if p in desired_products_list]
                    for product_name in relevant_products_for_nutrition:
                        if product_name in self.nutritional_info_df.index:
                            association_nutritional_score_sum += self.nutritional_info_df.loc[product_name, score_column_name]
                            nutritional_product_count += 1
                    if nutritional_product_count > 0:
                        candidate_associations_df.loc[index, 'normalized_nutritional_score'] = association_nutritional_score_sum / nutritional_product_count
        candidate_associations_df['final_score'] += candidate_associations_df['normalized_nutritional_score'] * user_preferences.get('weight_nutritional_goal', 0.15)

        # Score de Relevância Produtiva Regional
        candidate_associations_df['normalized_regional_production_score'] = 0.0
        if user_preferences.get('consider_regional_production_relevance', True) and desired_products_list:
            for index, row in candidate_associations_df.iterrows():
                association_regional_production_score_sum = 0
                regional_production_product_count = 0
                relevant_products_for_production = [p for p in row['produtos'] if p in desired_products_list]
                for product_name in relevant_products_for_production:
                    max_relevance_for_product = 0
                    for region_name in row['regioes']: # Associações podem atuar em múltiplas regiões
                        # Os nomes das regiões no df_producao devem corresponder (ex: ambos MAIÚSCULOS)
                        production_data_entry = self.regional_production_df[
                            (self.regional_production_df['regiao'] == region_name.upper()) & \
                            (self.regional_production_df['produto'] == product_name)
                        ]
                        if not production_data_entry.empty:
                            max_relevance_for_product = max(max_relevance_for_product, production_data_entry['relevancia_regiao_percent'].iloc[0])
                    association_regional_production_score_sum += (max_relevance_for_product / 100.0) # Normaliza para 0-1
                    regional_production_product_count += 1
                if regional_production_product_count > 0:
                    candidate_associations_df.loc[index, 'normalized_regional_production_score'] = association_regional_production_score_sum / regional_production_product_count
        candidate_associations_df['final_score'] += candidate_associations_df['normalized_regional_production_score'] * user_preferences.get('weight_regional_production_relevance', 0.20)
        
        # Score Colaborativo (Item-Item)
        candidate_associations_df['normalized_collaborative_score'] = 0.0
        if consumer_id in self.ratings_matrix_filled.index and not self.association_similarity_df.empty:
            candidate_associations_df['raw_collaborative_score'] = candidate_associations_df['id'].apply(
                lambda assoc_id: self._get_association_collaborative_score(consumer_id, assoc_id)
            )
            # Normaliza o score colaborativo (0-1) se houver scores > 0
            max_collab_score = candidate_associations_df['raw_collaborative_score'].max()
            if max_collab_score > 0:
                 candidate_associations_df['normalized_collaborative_score'] = candidate_associations_df['raw_collaborative_score'] / max_collab_score
            else:
                 candidate_associations_df['normalized_collaborative_score'] = 0.0 # Nenhum score colaborativo relevante
        candidate_associations_df['final_score'] += candidate_associations_df['normalized_collaborative_score'] * user_preferences.get('weight_collaborative_score', 0.20)

        # Seleciona as Top N recomendações
        top_n_recommendations = user_preferences.get('top_n_results', 5)
        final_recommendations_df = candidate_associations_df.sort_values(by='final_score', ascending=False).head(top_n_recommendations)

        columns_to_return = [
            'id', 'nome', 'distance_km', 'avaliacao_media', 'produtos', 'final_score',
            'normalized_distance_score', 'normalized_rating_score', 
            'normalized_nutritional_score', 'normalized_regional_production_score', 'normalized_collaborative_score'
        ]
        # Retorna apenas colunas que existem no DataFrame final
        existing_columns_to_return = [col for col in columns_to_return if col in final_recommendations_df.columns]
        return final_recommendations_df[existing_columns_to_return]

O dicionário `user_preferences` customiza a busca. Aqui estão os parâmetros aceitos:

* **`'desired_products'`**:
    * **Descrição**: Lista dos produtos específicos que o consumidor deseja encontrar.
    * **Tipo**: `List[str]` (Lista de strings)
    * **Valores Possíveis**: Cada string deve ser um nome de produto exatamente como definido em `produtos_escopo`
    * **Exemplos**:
        * `['Alface', 'Tomate']`
        * `['Morango']`
        * `['Abacate']`
        * `[]` (Lista vazia - o sistema não filtrará por produtos específicos, mas ainda considerará este campo para os scores nutricional e de relevância produtiva se preenchido.)
    * **Comportamento do Sistema**: O sistema buscará associações que ofereçam *pelo menos um* dos produtos listados.

* **`'max_distance_km'`**:
    * **Descrição**: A distância máxima (em quilômetros) que o consumidor está disposto a percorrer ou considerar.
    * **Tipo**: `int` ou `float`
    * **Valores Possíveis**: Um número positivo.
    * **Exemplos**: `10`, `15`, `20.5`, `50`
    * **Padrão no Sistema (se não fornecido)**: `30` km

* **`'only_organic'`**:
    * **Descrição**: Define se a busca deve ser restrita a associações com foco principal em produtos orgânicos.
    * **Tipo**: `bool` (Booleano)
    * **Valores Possíveis**:
        * `True`: Somente associações marcadas como `organico_principal = True`.
        * `False`: Considera todas as associações, independentemente de serem orgânicas.
    * **Padrão no Sistema (se não fornecido)**: `False`

* **`'nutritional_goal'`**:
    * **Descrição**: O objetivo nutricional principal para os produtos desejados. Influencia o `normalized_nutritional_score`.
    * **Tipo**: `str` (String) ou `None`
    * **Valores Possíveis (baseado nas colunas de score em `df_nutrientes`):**
        * `'alta_vitamina_c'` (para usar `score_vitamina_c`)
        * `'alta_fibra'` (para usar `score_fibras`)
        * `'baixa_caloria'` (para usar `score_baixa_caloria`)
        * `None` (Nenhum objetivo nutricional específico será usado para pontuação).

* **`'top_n'`**:
    * **Descrição**: O número máximo de associações recomendadas a serem retornadas.
    * **Tipo**: `int`
    * **Valores Possíveis**: Um número inteiro positivo.
    * **Exemplos**: `3`, `5`, `10`
    * **Padrão no Sistema (se não fornecido)**: `5` (conforme `preferencias.get('top_n', 5)` no método `recomendar`).

* **`'consider_regional_production_relevance'`**:
    * **Descrição**: Define se a relevância produtiva da região da associação (dados da EMATER) para os produtos desejados deve ser considerada no score.
    * **Tipo**: `bool` (Booleano)
    * **Valores Possíveis**:
        * `True`: O `normalized_regional_production_score` será calculado e considerado.
        * `False`: Este score não será considerado (ou será zero).
    * **Padrão no Sistema (se não fornecido)**: `True` (conforme `preferencias.get('considerar_relevancia_produtiva_regiao', True)`)

* **Pesos para os Componentes do Score Final**:
    * **Descrição**: Estes parâmetros controlam a importância relativa de cada fator no cálculo do `score_final`. Valores maiores indicam maior importância.
    * **Tipo**: `float` ou `int` (geralmente não negativos)
    * **Valores Possíveis**: Números como `0.0`, `0.1`, `0.25`, `0.5`, `1.0`, etc. A soma total dos pesos não precisa ser 1.
    * **Parâmetros de Peso e seus Padrões no Sistema (se não fornecidos, conforme o método `recomendar`):**
        * **`'weight_distance'`**: Padrão `0.25`
        * **`'weight_rating'`**: Padrão `0.20`
        * **`'weight_nutritional_goal'`**: Padrão `0.15`
        * **`'weight_regional_production_relevance'`**: Padrão `0.20`
        * **`'weight_collaborative_score'`**: Padrão `0.20`
    * **Exemplo de como usá-los no seu dicionário de preferências:**
        ```python
        'weight_distance': 0.30,
        'weight_rating': 0.15,
        'weight_nutritional_goal': 0.10,
        'weight_regional_production_relevance': 0.20,
        'weight_collaborative_score': 0.25
        ```

# Gerando mapas com o folium

A função `criar_mapa_recomendacoes_folium_v2` gera um mapa interativo para visualizar as associações recomendadas em relação à localização do usuário.

### Legenda de Cores e Ícones dos Marcadores

* **📍 Sua Localização (Usuário)**:
    * **Cor**: Vermelho (`red`)
    * **Ícone**: `user` (representa uma pessoa)
    * **Descrição**: Marca o ponto de latitude e longitude fornecido como a localização atual do usuário.

* **⭐ Associações Recomendadas**:
    * **Cor**:
        * Verde (`green`): Indica que a associação tem como foco principal produtos orgânicos (ou seja, a coluna `organico_principal` da associação é `True`).
        * Laranja (`orange`): Indica que a associação NÃO tem como foco principal produtos orgânicos (ou seja, `organico_principal` é `False`).
    * **Ícone**: `shopping-basket` (representa uma cesta de compras)
    * **Descrição**: Cada marcador representa uma associação que foi recomendada pelo sistema.

* **Raio de Busca**:
    * **Cor**: Azul (`blue`)
    * **Descrição**: Um círculo translúcido desenhado ao redor da localização do usuário. O raio deste círculo corresponde ao valor de `distancia_max_km` que foi definido nas preferências da busca, indicando a área geográfica considerada para as recomendações.

### Informações no Popup do Marcador

Ao clicar em um marcador de uma associação recomendada no mapa, um popup (janela de informações) é exibido com os seguintes detalhes:

* **Nome da Associação**: O nome da cooperativa ou associação.
* **Score Final**: A pontuação final (de 0 a 1, ou conforme a soma dos pesos) que a associação recebeu do sistema de recomendação.
* **Distância**: A distância em quilômetros da associação em relação à localização do usuário.
* **Avaliação Média**: A avaliação média da associação (conforme os dados em `df_associacoes`).
* **Foco em Orgânicos**: "Sim" se `organico_principal` for `True`, "Não" caso contrário.
* **Principais Produtos**: Uma amostra dos produtos oferecidos pela associação.
* **Detalhes dos Scores**: Uma seção expansível (`<details>`) que mostra os valores individuais dos sub-scores normalizados (Distância, Avaliação, Nutricional, Relevância Produtiva Regional, Colaborativo) que compuseram o `score_final`. Isso ajuda a entender por que uma associação específica foi bem pontuada.

In [61]:
def criar_mapa_recomendacoes(
    user_lat, user_lon,
    raw_recommendations_df,       # DataFrame de recomendações (SAÍDA do método recomendar)
    all_associations_data_df,   # SEU df_associacoes original e completo
    map_preferences_dict        # Dicionário de preferências usado na busca (para o raio)
):
    if pd.isna(user_lat) or pd.isna(user_lon):
        print("Coordenadas do usuário inválidas para o mapa.")
        return None
    
    if raw_recommendations_df.empty:
        # Este print é opcional, pois o fluxo principal já informa se não há recomendações
        # print("Nenhuma recomendação para exibir no mapa (DataFrame de recomendações vazio).")
        return None

    # CORREÇÃO AQUI: Merge para adicionar APENAS as colunas necessárias de all_associations_data_df
    # que não estão em raw_recommendations_df.
    # raw_recommendations_df JÁ CONTÉM: 'id', 'nome', 'dist_km', 'avaliacao_media', 'produtos',
    # 'score_final', 's_dist', 's_aval', 's_nutri', 's_relev_prod', 's_collab'.
    # Precisamos adicionar: 'latitude', 'longitude', 'organico_principal'.
    df_recommendations_with_coords = pd.merge(
        raw_recommendations_df,
        all_associations_data_df[['id', 'latitude', 'longitude', 'organico_principal']], # Seleciona apenas estas de df_associacoes
        on='id',  # Chave para o merge
        how='left' # Mantém todas as linhas de raw_recommendations_df
    )
    
    mapa = folium.Map(location=[user_lat, user_lon], zoom_start=10)
    folium.Marker(
        [user_lat, user_lon],
        popup="Sua Localização",
        tooltip="Você",
        icon=folium.Icon(color='red', icon='user', prefix='fa')
    ).add_to(mapa)
    
    dist_max_km = map_preferences_dict.get('max_distance_km', 30)
    folium.Circle(
        [user_lat, user_lon],
        radius=dist_max_km * 1000,
        color='blue',
        fill=True,
        fill_opacity=0.1,
        tooltip=f"Raio: {dist_max_km} km"
    ).add_to(mapa)

    # As colunas 'nome', 'score_final', 'dist_km', 'avaliacao_media', 'produtos',
    # 'organico_principal' (após o merge), e todas as 's_...' já estão em df_recommendations_with_coords.
    for _, row in df_recommendations_with_coords.iterrows():
        # Verifica se latitude e longitude foram corretamente adicionadas e não são NaN
        if pd.notna(row['latitude']) and pd.notna(row['longitude']):
            prods_list = row.get('produtos', [])
            prods_str = ", ".join(prods_list[:3]) + ('...' if len(prods_list) > 3 else '')
            
            popup_html = f"""<b>{row['nome']}</b><hr>
                             <b>Score:</b> {row.get('score_final', 0.0):.2f}<br>
                             <b>Distância:</b> {row.get('dist_km', 0.0):.1f} km<br>
                             <b>Avaliação:</b> {row.get('avaliacao_media', 0.0):.1f}/5<br>
                             <b>Orgânico:</b> {'Sim' if row.get('organico_principal', False) else 'Não'}<br>
                             <b>Produtos:</b> {prods_str}<br>
                             <details><summary>Sub-scores</summary><small>
                             Dist: {row.get('s_dist', 0):.2f} | Aval: {row.get('s_aval', 0):.2f} <br>
                             Nutri: {row.get('s_nutri', 0):.2f} | Relev: {row.get('s_relev_prod', 0):.2f} | Collab: {row.get('s_collab', 0):.2f}
                             </small></details>"""
            folium.Marker(
                [row['latitude'], row['longitude']],
                popup=folium.Popup(popup_html, max_width=380),
                tooltip=f"{row['nome']} (Score: {row.get('score_final',0):.2f})",
                icon=folium.Icon(color='green' if row.get('organico_principal', False) else 'orange', icon='shopping-basket', prefix='fa')
            ).add_to(mapa)
    return mapa

In [59]:
recommendation_system = SistemaRecomendacaoDF(df_associacoes, df_nutrientes, df_producao, df_utility_pivot)

Matriz de similaridade Associação-Associação calculada ((17, 17)).
SistemaRecomendacaoDF inicializado.


In [68]:
print("\n--- CENÁRIO DE EXEMPLO 1 ---")
example1_user_latitude, example1_user_longitude = -15.7632, -47.8706
print(len(df_utility_pivot))
example1_consumer_id = df_utility_pivot.index[np.random.randint(0, len(df_utility_pivot))]

example1_user_preferences = {
    'produtos_desejados': ['Alface', 'Tomate'],
    'max_distance_km': 25,
    'apenas_organicos': False,
    'objetivo_nutricional': 'baixa_caloria', 
    'top_n': 5,
    'considerar_relevancia_produtiva_regiao': True,
    'peso_distancia': 0.25, 'peso_avaliacao': 0.20,
    'peso_nutricional': 0.20, 'peso_relevancia_prod': 0.20,
    'peso_colaborativo': 0.15
}

print(f"Solicitando recomendações para: {example1_consumer_id}")
recommendations_scenario1 = recommendation_system.recomendar(
    example1_consumer_id, example1_user_latitude, example1_user_longitude, example1_user_preferences
)

if not recommendations_scenario1.empty:
    print(f"\nRecomendações para o Cenário 1 (Usuário: {example1_consumer_id}):")
    display(recommendations_scenario1)
    print("\nGerando mapa para o Cenário 1...")
    map_scenario1 = criar_mapa_recomendacoes(
        example1_user_latitude, example1_user_longitude,
        recommendations_scenario1,
        df_associacoes,
        example1_user_preferences
    )
    if map_scenario1:
        display(map_scenario1)
else:
    print(f"Nenhuma recomendação encontrada para o Cenário 1 (Usuário: {example1_consumer_id}).")


--- CENÁRIO DE EXEMPLO 1 ---
500
Solicitando recomendações para: Consumidor_178

🔄 Gerando recomendações para 'Consumidor_178' em (-15.7632, -47.8706)
Preferências: N/A, Dist. Max: 25km
🔎 3 associações candidatas após filtros iniciais.

Recomendações para o Cenário 1 (Usuário: Consumidor_178):


,id,nome,distance_km,avaliacao_media,produtos,final_score,normalized_distance_score,normalized_rating_score,normalized_nutritional_score,normalized_regional_production_score,normalized_collaborative_score
5,6,ASPHOR - Hortigranjeiros DF,8.909386,3.572727,"[Abacate, Abóbora, Couve, Alface, Batata, Bete...",0.592347,1.000000,0.714545,0.0,0.0,0.997192
0,1,AFECA - Assentamento 15 de Agosto,22.899497,4.405323,"[Abóbora, Couve, Alface, Cebola, Chuchu, Bróco...",0.390123,0.085927,0.881065,0.0,0.0,0.962144
3,4,ASPAF - Produtores Agricultura Familiar,24.214628,4.527959,"[Goiaba, Morango, Tangerina, Abacate, Abóbora,...",0.381118,0.000000,0.905592,0.0,0.0,1.000000



Gerando mapa para o Cenário 1...


In [69]:
print("\n--- CENÁRIO DE EXEMPLO 2 ---")
example2_user_latitude, example2_user_longitude = -15.8375, -47.9123 # Sudoeste/Octogonal
example2_consumer_id = df_utility_pivot.index[np.random.randint(0, len(df_utility_pivot))]

example2_user_preferences = {
    'produtos_desejados': ['Abacate'],
    'max_distance_km': 15,
    'apenas_organicos': True,
    'objetivo_nutricional': None,
    'top_n': 3, # Chave correta
    'considerar_relevancia_produtiva_regiao': False,
    'peso_distancia': 0.30,
    'peso_avaliacao': 0.25,
    'peso_nutricional': 0.10,
    'peso_relevancia_prod': 0.10,
    'peso_colaborativo': 0.25
}
print(f"Solicitando recomendações para: {example2_consumer_id}")
recommendations_scenario2 = recommendation_system.recomendar(
    example2_consumer_id, example2_user_latitude, example2_user_longitude, example2_user_preferences
)

if not recommendations_scenario2.empty:
    print(f"\nRecomendações para o Cenário 2 (Usuário: {example2_consumer_id}):")
    display(recommendations_scenario2)
    print("\nGerando mapa para o Cenário 2...")
    map_scenario2 = criar_mapa_recomendacoes(
        example2_user_latitude, example2_user_longitude,
        recommendations_scenario2,
        df_associacoes,
        example2_user_preferences
    )
    if map_scenario2:
        display(map_scenario2)
else:
    print(f"Nenhuma recomendação encontrada para o Cenário 2 (Usuário: {example2_consumer_id}).")


--- CENÁRIO DE EXEMPLO 2 ---
Solicitando recomendações para: Consumidor_326

🔄 Gerando recomendações para 'Consumidor_326' em (-15.8375, -47.9123)
Preferências: N/A, Dist. Max: 15km
🔎 1 associações candidatas após filtros iniciais.

Recomendações para o Cenário 2 (Usuário: Consumidor_326):


,id,nome,distance_km,avaliacao_media,produtos,final_score,normalized_distance_score,normalized_rating_score,normalized_nutritional_score,normalized_regional_production_score,normalized_collaborative_score
5,6,ASPHOR - Hortigranjeiros DF,6.402045,3.572727,"[Abacate, Abóbora, Couve, Alface, Batata, Bete...",0.592909,1.0,0.714545,0.0,0.0,1.0



Gerando mapa para o Cenário 2...


In [70]:
print("\n--- CENÁRIO DE EXEMPLO 3: Consumidor Customizado ---")

# 1. Definições para o consumidor customizado
custom_consumer_id = "Consumidor_Gourmet_001"
custom_ratings_list = [
    {'id_associacao': 3, 'avaliacao': 5},  # AMISTA (Orgânicos)
    {'id_associacao': 15, 'avaliacao': 4}, # Rede Terra (Ecológicos)
    {'id_associacao': 6, 'avaliacao': 2}   # ASPHOR
]

# 2. Criar uma matriz de utilidade específica para este cenário, incluindo o consumidor customizado
print(f"Configurando dados para o consumidor: {custom_consumer_id}")
df_utility_pivot_cen3 = df_utility_pivot.copy() # Começa com a matriz de utilidade base
custom_ratings_series_cen3 = pd.Series(
    data=[r['avaliacao'] for r in custom_ratings_list],
    index=[r['id_associacao'] for r in custom_ratings_list],
    name=custom_consumer_id
)

# Adiciona ou atualiza o consumidor customizado na cópia da matriz de utilidade
if custom_consumer_id in df_utility_pivot_cen3.index:
    df_utility_pivot_cen3.loc[custom_consumer_id] = custom_ratings_series_cen3.reindex(df_utility_pivot_cen3.columns).fillna(np.nan)
else:
    new_row_df = pd.DataFrame([custom_ratings_series_cen3], index=[custom_consumer_id])
    new_row_df = new_row_df.reindex(columns=df_utility_pivot_cen3.columns).fillna(np.nan)
    df_utility_pivot_cen3 = pd.concat([df_utility_pivot_cen3, new_row_df])

# 3. Instanciar um sistema de recomendação que use esta matriz de utilidade customizada
custom_recommendation_system = SistemaRecomendacaoDF(
    df_associacoes, df_nutrientes, df_producao, df_utility_pivot_cen3
)

# 4. Definir localização e preferências para o consumidor customizado
custom_user_latitude_cen3, custom_user_longitude_cen3 = -15.8271, -47.8303 # Lago Sul
custom_user_preferences_cenario3 = {
    'produtos_desejados': ['Alface', 'Tomate', 'Agrião', 'Morango'],
    'max_distance_km': 20,
    'apenas_organicos': True,
    'objetivo_nutricional': 'alta_vitamina_c',
    'top_n': 3,
    'considerar_relevancia_produtiva_regiao': True,
    'weight_distance': 0.20,
    'weight_rating': 0.25,
    'weight_nutricional_goal': 0.15,
    'weight_regional_production_relevance': 0.10,
    'weight_collaborative_score': 0.30
}

print(f"\nSolicitando recomendações para o consumidor customizado: {custom_consumer_id}")
recommendations_scenario3 = custom_recommendation_system.recomendar(
    custom_consumer_id,
    custom_user_latitude_cen3,
    custom_user_longitude_cen3,
    custom_user_preferences_cenario3
)

if not recommendations_scenario3.empty:
    print(f"\nRecomendações para o Cenário 3 (Consumidor Customizado: {custom_consumer_id}):")
    display(recommendations_scenario3)
    print("\nGerando mapa para o Cenário 3...")
    map_scenario3 = criar_mapa_recomendacoes(
        custom_user_latitude_cen3,
        custom_user_longitude_cen3,
        recommendations_scenario3,
        df_associacoes,
        custom_user_preferences_cenario3
    )
    if map_scenario3:
        display(map_scenario3)
else:
    print(f"Nenhuma recomendação encontrada para o Cenário 3 (Consumidor Customizado: {custom_consumer_id}).")


--- CENÁRIO DE EXEMPLO 3: Consumidor Customizado ---
Configurando dados para o consumidor: Consumidor_Gourmet_001
Matriz de similaridade Associação-Associação calculada ((17, 17)).
SistemaRecomendacaoDF inicializado.

Solicitando recomendações para o consumidor customizado: Consumidor_Gourmet_001

Gerando recomendações para 'Consumidor_Gourmet_001' em (-15.8271, -47.8303)
Preferências: N/A, Dist. Max: 20km
3 associações candidatas após filtros iniciais.

Recomendações para o Cenário 3 (Consumidor Customizado: Consumidor_Gourmet_001):


,id,nome,distance_km,avaliacao_media,produtos,final_score,normalized_distance_score,normalized_rating_score,normalized_nutritional_score,normalized_regional_production_score,normalized_collaborative_score
5,6,ASPHOR - Hortigranjeiros DF,13.260213,3.572727,"[Abacate, Abóbora, Couve, Alface, Batata, Bete...",0.675198,1.000000,0.714545,0.0,0.0,0.988539
0,1,AFECA - Assentamento 15 de Agosto,14.937004,4.405323,"[Abóbora, Couve, Alface, Cebola, Chuchu, Bróco...",0.583783,0.380835,0.881065,0.0,0.0,0.957834
3,4,ASPAF - Produtores Agricultura Familiar,15.968363,4.527959,"[Goiaba, Morango, Tangerina, Abacate, Abóbora,...",0.526398,0.000000,0.905592,0.0,0.0,1.000000



Gerando mapa para o Cenário 3...
